# P. Visión

In [1]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure

In [2]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

In [3]:
path = 'C:\\Users\\alumno\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain = leerFotos(path)

path = 'C:\\Users\\alumno\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

In [ ]:
def generaImagenesRecortadas(dataPos,dataNeg):    
    numeroAleatoriasPorFoto = 3       # Numero de recortadas aleatorias por foto
    rowsFotoPersona = dataPos[0].shape[0]
    colsFotoPersona = dataPos[0].shape[1]

    rowsFotoEntorno = dataNeg[0].shape[0]
    colsFotoEntorno = dataNeg[0].shape[1]

    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        for j in range(numeroAleatoriasPorFoto):
            randRow = random.randint(0,rowsFotoEntorno - rowsFotoPersona)
            randCol = random.randint(0,colsFotoEntorno - colsFotoPersona)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona,randCol: randCol + colsFotoPersona]
            fotosNegTrain[(i,j)] = randomImage 
    return fotosNegTrain

In [ ]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

In [ ]:
nFilasFoto,nColsFoto = fotosPosTrain[0].shape
print(nFilasFoto/2)

In [ ]:
fdPosTrain = np.zeros((len(fotosPosTrain),6804))
for i in range(len(fotosPosTrain)):
#fd,imHog = hog(fotosPosTrain[1], orientations=8, pixels_per_cell=(8, 8),cells_per_block=(3, 3),visualize=True)
    fdPosTrain[i] = hog(fotosPosTrain[i])
    #imHog_re = exposure.rescale_intensity(imHog, in_range = (0,10))
print(fdPosTrain.shape,fotosPosTrain[1].shape,134*70)

In [ ]:
# fdNegTrain = np.zeros((len(fotosNegTrain),6804))
# for i in range(len(fotosPosTrain)):
#     fdNegTrain[i] = hog(fotosPosTrain[i])
fdTest,imHogTest = hog(fotosNegTrain[1], orientations=9, pixels_per_cell=(8, 8),cells_per_block=(3, 3),visualize=True)
print(fdTest.shape,imHogTest.shape)

In [ ]:
cv2.imshow("sadas",imHogTest)
cv2.imshow("ds2da",fotosNegTrain[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
print()